In [8]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline
%load_ext autoreload
%autoreload 2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from caveclient import CAVEclient
import pandas as pd
import numpy as np
import torch
import json
from pathlib import Path
from datetime import timedelta

In [12]:
# Initialize CAVE client
datastack_name = "minnie65_phase3_v1"
my_token = "64df9664ce8a28852ee99167c26a9e8d"
client = CAVEclient(datastack_name=datastack_name, auth_token=my_token)

In [13]:
mat_timestamp = client.materialize.get_timestamp(version=943)
mat_version = 943
# [date, timestamp, mean_coord_x, mean_coord_y, mean_coord_z, operation_id]
df = pd.read_feather('../../data/minnie_splitlog_240927/minnie_splitlog_240927.feather')
# [operation_id, sink supervoxel id, source_supervoxel id]
edits = np.load('../../data/minnie_splitlog_240927/minnie_splitlog_240927.npy')
save_directory = f"../../data/pruned_minnie_splitlog_{mat_version}"

In [14]:
import data_utils
data_utils.prune_edits_before_mat(client, 943, edits, df, save_directory)

KeyboardInterrupt: 

In [ ]:
# Below is testing

In [21]:
print(mat_timestamp)

2024-01-22 08:10:01.497934+00:00


In [19]:
df = df.drop(df[df['date'] > mat_timestamp].index)

In [47]:
import os
mat_version = 943
pruned_directory = f"../../data/pruned_minnie_splitlog_{mat_version}"
if not os.path.exists(pruned_directory):
    os.makedirs(pruned_directory)
df.to_feather(f"{pruned_directory}/splitlog_{mat_version}.feather")

In [22]:
df.tail(10)

,date,timestamp,mean_coord_x,mean_coord_y,mean_coord_z,operation_id
496277,2024-01-21 16:02:43.092000+00:00,1.705853e+09,125098.633333,108029.577778,26699.00,903967
496278,2024-01-21 16:10:44.107000+00:00,1.705853e+09,106958.750000,87577.250000,19572.00,903969
496279,2024-01-21 16:11:07.639000+00:00,1.705853e+09,106821.000000,87684.666667,19573.00,903970
496280,2024-01-21 16:11:30.140000+00:00,1.705853e+09,106862.600000,87801.900000,19574.50,903971
496281,2024-01-21 16:12:25.789000+00:00,1.705854e+09,128587.750000,95285.000000,20869.00,903975
496282,2024-01-21 16:14:40.675000+00:00,1.705854e+09,128785.500000,105198.500000,19279.00,903977
496283,2024-01-21 16:16:16.363000+00:00,1.705854e+09,122913.000000,101388.833333,23303.50,903978
496284,2024-01-21 16:17:36.308000+00:00,1.705854e+09,122908.250000,106068.750000,20483.25,903980
496285,2024-01-21 16:18:12.197000+00:00,1.705854e+09,125220.000000,117978.250000,21464.50,903981
496286,2024-01-22 07:53:26.939000+00:00,1.705910e+09,98713.750000,90413.000000,21808.75,903984


In [30]:
prune_indices = []
op_ids = set(df['operation_id'])
for index, edit in enumerate(edits):
    if edit[0] in op_ids:
        prune_indices.append(index)

In [33]:
print(len(prune_indices))
print(prune_indices[0:10])
print(prune_indices[-10:])

176204546
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[176204536, 176204537, 176204538, 176204539, 176204540, 176204541, 176204542, 176204543, 176204544, 176204545]


In [36]:
print(edits[176204545])
print(edits[176204544])
print(edits[176204543])

[           903984 91917661126398025 91917661126398053]
[           903984 91917661126398025 91917661126398053]
[           903981 99173063413198166 99173063413198174]


In [37]:
prune_edits = edits[prune_indices]

In [38]:
print(prune_edits.shape)
print(prune_edits[0:10])
print(prune_edits[-10:])

(176204546, 3)
[[            47222 97197996596845829 97197996596845891]
 [            47222 97197996596845829 97197996596851211]
 [            47222 97197996596845829 97197996596845891]
 [            47222 97197996596845829 97197996596851211]
 [           105222 98393028632722115 98393028632725107]
 [           105222 98393028632725065 98393028632726303]
 [           105222 98393028632725065 98393028632734564]
 [           105222 98393028632725107 98393028632726303]
 [           105222 98393028632725107 98393028632728301]
 [           105222 98393028632726303 98393028632727324]]
[[           903981 99243432157365429 99173063413198174]
 [           903981 99243432157365429 99243432157366481]
 [           903981 99173063413198166 99173063413198174]
 [           903981 99173063413198174 99243432157365429]
 [           903981 99173063413198174 99243432157365429]
 [           903981 99243432157365429 99173063413198174]
 [           903981 99243432157365429 99243432157366481]
 [           90

In [9]:
counter = 0
for op_id in df['operation_id']:
    print(op_id)
    counter += 1
    if counter == 10:
        break

47222
105222
107815
115686
115687
115689
115690
115691
115700
115703


In [48]:
np.save(f"{pruned_directory}/splitlog_{mat_version}.npy", prune_edits)